In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re, random

In [2]:
def trim(string):
    if not string:
        return None
    
    string = string.strip()\
                   .replace('\r', '')\
                   .replace('\r', '')
    
    return string

In [3]:
cities_page_url = 'https://www.yelp.com/city'
city_urls = []
city_names = []

city_page_html = urlopen(cities_page_url).read().decode('utf-8')
soup = BeautifulSoup(city_page_html, features='lxml')
city_href_links = soup.find_all('a', {'href': re.compile('/city/.*?')})
for link in city_href_links:
    city_urls.append(link['href'])
    city_names.append(link.get_texJt())

print('\n'.join(city_urls))
print('\n'.join(city_names))
print('How many cities? ', len(city_names))

/city/birmingham-al
/city/huntsville-al-us
/city/chandler-az-us
/city/gilbert-az-us
/city/glendale-az-us
/city/mesa-az-us
/city/phoenix
/city/scottsdale-az-us
/city/tempe-az-us
/city/tucson-az-us
/city/anaheim-ca-us
/city/bakersfield
/city/berkeley
/city/chula-vista-ca-us
/city/concord-ca-us
/city/davis
/city/elk-grove-ca-us
/city/escondido-ca-us
/city/fairfield-ca-us
/city/fullerton-ca-us
/city/hollywood-ca-us
/city/lancaster-ca-us
/city/la
/city/modesto-ca-us
/city/moreno-valley-ca-us
/city/oakland
/city/north-county-san-diego
/city/pasadena-ca-us
/city/rancho-cucamonga-ca-us
/city/richmond-ca-us
/city/riverside-ca-us
/city/roseville-ca-us
/city/sacramento
/city/san-bernardino-ca-us
/city/san-diego
/city/sf
/city/san-jose
/city/santa-ana-ca-us
/city/santa-clarita-ca-us
/city/simi-valley-ca-us
/city/sunnyvale-ca-us
/city/torrance-ca-us
/city/west-covina-ca-us
/city/arvada-co-us
/city/aurora-co-us
/city/boulder
/city/ColoradoSprings
/city/denver
/city/fort-collins-co-us
/city/westminst

## scrape business urls for a specific city

In [24]:
base_url = 'https://www.yelp.com/search?find_loc='
biz_urls = []

target_city_name = 'Corpus Christi'
target_city_url = base_url + target_city_name.replace(' ', '+')

target_city_page_html = urlopen(target_city_url).read().decode('utf-8')
soup = BeautifulSoup(target_city_page_html, features='lxml')

biz_href_elems = soup.find_all('a', {
    'class': 'biz-name',
    'href': re.compile('/biz/.*?')
})

len(biz_href_elems)

for biz_href_elem in biz_href_elems:
    biz_urls.append(biz_href_elem['href'])
    
print('\n'.join(biz_urls))

/biz/andys-country-kitchen-corpus-christi
/biz/ginger-cafe-and-grill-corpus-christi
/biz/hesters-cafe-and-coffee-bar-six-points-corpus-christi
/biz/z-counter-corpus-christi-2
/biz/8te-corpus-christi
/biz/salt-water-grill-corpus-christi-2
/biz/big-bowl-korean-bbq-corpus-christi
/biz/the-blue-clove-corpus-christi-2
/biz/authentic-new-york-pizza-corpus-christi
/biz/thai-cottage-corpus-christi-corpus-christi-2


## scrape business info for a given business url

In [95]:
domain_url = 'https://www.yelp.com'
biz_url = domain_url + biz_urls[0]
biz_info = {}

biz_page_html = urlopen(biz_url).read().decode('utf-8')
soup = BeautifulSoup(biz_page_html, features='lxml')

biz_name = soup.find('h1', {'class': 'biz-page-title'}).get_text().strip()
biz_info['biz_name'] = biz_name

review_count = soup.find('span', {'class': 'review-count rating-qualifier'}).get_text()
review_count = int(review_count.strip().replace(' reviews', ''))
biz_info['review_count'] = review_count

tags_elems = soup.find('span', {'class': 'category-str-list'})\
           .find_all('a')
tags = [tag_elem.get_text() for tag_elem in tags_elems]
biz_info['tags'] = tags

address_elem = soup.find('address')
biz_info['address_street'] = address_elem.find('span', {'itemprop': 'streetAddress'}).get_text()
biz_info['address_state'] = address_elem.find('span', {'itemprop': 'addressRegion'}).get_text()
biz_info['address_postalcode'] = address_elem.find('span', {'itemprop': 'postalCode'}).get_text()


biz_phone = soup.find('span', {'class': 'biz-phone'}).get_text().strip()
biz_phone = biz_phone.replace('(', '')\
                     .replace(')', '')\
                     .replace('-', '')\
                     .replace(' ', '')
biz_info['biz_phone'] = biz_phone
            
biz_info['more_biz_info'] = {}
more_biz_info_elems = soup.find('div', {'class': 'short-def-list'}).find_all('dl')
for biz_info_elem in more_biz_info_elems:
    key = trim(biz_info_elem.find('dt').get_text())
    value = trim(biz_info_elem.find('dd').get_text())
    biz_info['more_biz_info'][key] = value

for key, value in biz_info.items():
    if key == 'more_biz_info':
        for subkey, subvalue in value.items():
            print('-- ', subkey, ': ', subvalue)
    else:
        print(key, ': ', value)


biz_name :  Andy’s Country Kitchen
review_count :  428
tags :  ['Southern', 'Breakfast & Brunch', 'American (Traditional)']
address_street :  5802 S Staples St
address_state :  TX
address_postalcode :  78413
biz_phone :  3619930251
--  Liked by Vegans :  Yes
--  Liked by Vegetarians :  Yes
--  Takes Reservations :  No
--  Delivery :  No
--  Take-out :  Yes
--  Accepts Credit Cards :  Yes
--  Accepts Apple Pay :  No
--  Accepts Google Pay :  No
--  Good For :  Breakfast, Brunch
--  Parking :  Private Lot
--  Bike Parking :  Yes
--  Good for Kids :  Yes
--  Good for Groups :  Yes
--  Attire :  Casual
--  Ambience :  Casual
--  Noise Level :  Average
--  Alcohol :  No
--  Outdoor Seating :  No
--  Wi-Fi :  Free
--  Has TV :  No
--  Waiter Service :  Yes
--  Caters :  No


## Real practice scraping on Yelp
    

In [116]:
def get_yelp_city_names():
    cities_page_url = domain_url + '/city'
    city_names = []

    city_page_html = urlopen(cities_page_url).read().decode('utf-8')
    soup = BeautifulSoup(city_page_html, features='lxml')
    city_href_links = soup.find_all('a', {'href': re.compile('/city/.*?')})
    for link in city_href_links:
        city_urls.append(link['href'])
        city_names.append(link.get_text())

    return city_names


In [117]:
def get_biz_urls(city_name, page_number):
    base_url_template = 'https://www.yelp.com/search?find_loc=%s&start=%d'
    biz_urls = []

    target_city_url = base_url_template % (target_city_name.replace(' ', '+'), page_number)

    target_city_page_html = urlopen(target_city_url).read().decode('utf-8')
    soup = BeautifulSoup(target_city_page_html, features='lxml')

    biz_href_elems = soup.find_all('a', {
        'class': 'biz-name',
        'href': re.compile('/biz/.*?')
    })

    for biz_href_elem in biz_href_elems:
        biz_urls.append(biz_href_elem['href'])

    return biz_urls



In [120]:
def get_biz_info(biz_url):
    biz_url = domain_url + biz_url
    biz_info = {}

    biz_page_html = urlopen(biz_url).read().decode('utf-8')
    soup = BeautifulSoup(biz_page_html, features='lxml')

    biz_name = soup.find('h1', {'class': 'biz-page-title'}).get_text().strip()
    biz_info['biz_name'] = biz_name

    review_count = soup.find('span', {'class': 'review-count rating-qualifier'}).get_text()
    review_count = int(review_count.strip().replace(' reviews', ''))
    biz_info['review_count'] = review_count

    tags_elems = soup.find('span', {'class': 'category-str-list'})\
               .find_all('a')
    tags = [tag_elem.get_text() for tag_elem in tags_elems]
    biz_info['tags'] = tags

    address_elem = soup.find('address')
    biz_info['address_street'] = address_elem.find('span', {'itemprop': 'streetAddress'}).get_text()
    biz_info['address_state'] = address_elem.find('span', {'itemprop': 'addressRegion'}).get_text()
    biz_info['address_postalcode'] = address_elem.find('span', {'itemprop': 'postalCode'}).get_text()


    biz_phone = soup.find('span', {'class': 'biz-phone'}).get_text().strip()
    biz_phone = biz_phone.replace('(', '')\
                         .replace(')', '')\
                         .replace('-', '')\
                         .replace(' ', '')
    biz_info['biz_phone'] = biz_phone

    biz_info['more_biz_info'] = {}
    more_biz_info_elems = soup.find('div', {'class': 'short-def-list'}).find_all('dl')
    for biz_info_elem in more_biz_info_elems:
        key = trim(biz_info_elem.find('dt').get_text())
        value = trim(biz_info_elem.find('dd').get_text())
        biz_info['more_biz_info'][key] = value

#     for key, value in biz_info.items():
#         if key == 'more_biz_info':
#             for subkey, subvalue in value.items():
#                 print('-- ', subkey, ': ', subvalue)
#         else:
#             print(key, ': ', value)
            
    return biz_info

In [126]:
domain_url = 'https://www.yelp.com'
biz_info_dict = {}


# scrape first page of business info for each city
# one page contains 10 business info
for city_name in get_yelp_city_names():
    urls = get_biz_urls(city_name, 1)
    for biz_url in urls:
        biz_info = get_biz_info(biz_url)
        biz_info_dict[biz_info['biz_name']] = biz_info
        print('[Done] Scraping ', biz_info['biz_name'], biz_url)


KeyboardInterrupt: 

In [125]:
print(biz_info_dict)   

{'Ginger Cafe And Grill': {'biz_name': 'Ginger Cafe And Grill', 'review_count': 335, 'tags': ['Lebanese', 'Indian', 'Mediterranean'], 'address_street': '7009 S Staples St', 'address_state': 'TX', 'address_postalcode': '78413', 'biz_phone': '3619469834', 'more_biz_info': {'Liked by Vegans': 'Yes', 'Liked by Vegetarians': 'Yes', 'Takes Reservations': 'No', 'Delivery': 'No', 'Take-out': 'Yes', 'Accepts Credit Cards': 'Yes', 'Accepts Apple Pay': 'No', 'Accepts Google Pay': 'No', 'Good For': 'Lunch, Dinner', 'Parking': 'Private Lot', 'Bike Parking': 'Yes', 'Good for Kids': 'Yes', 'Good for Groups': 'Yes', 'Attire': 'Casual', 'Ambience': 'Casual', 'Noise Level': 'Average', 'Alcohol': 'No', 'Outdoor Seating': 'No', 'Wi-Fi': 'No', 'Has TV': 'No', 'Waiter Service': 'Yes', 'Caters': 'Yes'}}, 'Hester’s Cafe & Coffee Bar - Six Points': {'biz_name': 'Hester’s Cafe & Coffee Bar - Six Points', 'review_count': 355, 'tags': ['Coffee & Tea', 'Breakfast & Brunch', 'Sandwiches'], 'address_street': '1714 S